In [1]:
import pandas as pd
from io import StringIO

### Carregando o dataset

In [2]:
def load_dataset_from_json(path: str, num_lines: int = 1000) -> pd.DataFrame:
    """
    Load a dataset from a JSON file into a pandas DataFrame.

    Parameters:
    path: The file path of the JSON file to be read
    num_lines:: The maximum number of lines to read from the file

    Returns:
    A pandas DataFrame containing the parsed data from the JSON file
    """

    lines = []
    with open(path, 'r') as file:
        for i, line in enumerate(file):
            if i >= num_lines:
                break
            lines.append(line.strip())
    df = pd.read_json(StringIO('[' + ','.join(lines) + ']'), lines=False).reset_index(drop=True)
    return df

In [3]:
num_lines = 500000

df_authors = load_dataset_from_json('data/goodreads_book_authors.json', num_lines)
df_genres = load_dataset_from_json('data/goodreads_book_genres_initial.json', num_lines)
df_books = load_dataset_from_json('data/goodreads_books.json', num_lines)

In [4]:
df_interactions = pd.read_csv('data/goodreads_interactions.csv', nrows=50000000)

### Merge dos datasets carregados para criar um dataset mais completo

O primeiro passo para a criação de um dataset mais completo é remover as features irrelevantes e renomear as features que possivelmente serão utilizadas futuramente para a criação de um modelo preditivo

#### Autores

In [5]:
df_authors.drop(['name'], axis=1, inplace=True)

df_authors.rename(columns={
    'average_rating': 'author_rating',
    'text_reviews_count': 'author_reviews_count',
    'ratings_count': 'author_ratings_count'},inplace=True)

In [6]:
df_authors.head()

,author_rating,author_id,author_reviews_count,author_ratings_count
0,3.98,604031,7,49
1,4.08,626222,28716,546796
2,3.92,10333,5075,122118
3,3.68,9212,36262,888522
4,3.82,149918,96,1740


In [7]:
# Verificando os tipos das colunas do dataframe
df_authors.dtypes

author_rating           float64
author_id                 int64
author_reviews_count      int64
author_ratings_count      int64
dtype: object

#### Gêneros

Para a simplificação do modelo, o gênero a ser considerado de cada livro vai ser apenas o gênero principal do livro, ou seja, o primeiro gênero presente na coluna

In [8]:
df_genres['genres'] = df_genres['genres'].apply(lambda x: next(iter(x.keys())) if isinstance(x, dict) and x else None)

In [9]:
df_genres.rename(columns={'genres': 'book_genre'},inplace=True)

In [10]:
df_genres.head()

,book_id,book_genre
0,5333265,"history, historical fiction, biography"
1,1333909,fiction
2,7327624,"fantasy, paranormal"
3,6066819,fiction
4,287140,non-fiction


In [11]:
# Verificando os tipos das colunas do dataframe
df_genres.dtypes

book_id        int64
book_genre    object
dtype: object

#### Livros

Assim como nos gêneros de cada livro, o autor considerado vai ser apena o autor principal, ou seja, o primeiro autor presente na coluna

In [12]:
df_books.drop(['isbn', 'series', 'popular_shelves', 'asin', 'kindle_asin', 'description', 'link', 'publisher', 'publication_day',
               'isbn13', 'publication_month', 'similar_books', 'edition_information', 'url', 'image_url', 'work_id',
               'title', 'country_code'], axis=1, inplace=True)

df_books['authors'] = df_books['authors'].apply(lambda x: x[0]['author_id'] if isinstance(x, list) and len(x) > 0 else None)

In [13]:
df_books.rename(columns={
    'authors': 'author_id',
    'text_reviews_count': 'book_text_reviews_count',
    'average_rating': 'book_rating',
    'format': 'book_format',
    'ratings_count': 'book_ratings_count',
    'title_without_series': 'book_title'
    }, inplace=True)

In [14]:
df_books.head()

,book_text_reviews_count,language_code,is_ebook,book_rating,book_format,author_id,num_pages,publication_year,book_id,book_ratings_count,book_title
0,1,,false,4.00,Paperback,604031,256,1984,5333265,3,W.C. Fields: A Life on Film
1,6,,false,3.23,Audio CD,626222,,2001,1333909,10,Good Harbor
2,7,eng,false,4.03,Hardcover,10333,600,1987,7327624,140,"The Unschooled Wizard (Sun Wolf and Starhawk, ..."
3,3282,eng,false,3.49,Hardcover,9212,368,2009,6066819,51184,Best Friends Forever
4,5,,false,3.40,,149918,,,287140,15,Runic Astrology: Starcraft and Timekeeping in ...


In [15]:
# Verificando os tipos das colunas do dataframe
df_books.dtypes

book_text_reviews_count    object
language_code              object
is_ebook                   object
book_rating                object
book_format                object
author_id                  object
num_pages                  object
publication_year           object
book_id                     int64
book_ratings_count         object
book_title                 object
dtype: object

Além disso, é interessante alterar o tipo deas colunas que possuem valor numérico

In [16]:
df_books['book_text_reviews_count'] = pd.to_numeric(df_books['book_text_reviews_count'], errors='coerce').astype('Int64')
df_books['book_rating'] = pd.to_numeric(df_books['book_rating'], errors='coerce').astype('float64')
df_books['author_id'] = pd.to_numeric(df_books['author_id'], errors='coerce').astype('Int64')
df_books['num_pages'] = pd.to_numeric(df_books['num_pages'], errors='coerce').astype('Int64')
df_books['publication_year'] = pd.to_numeric(df_books['publication_year'], errors='coerce').astype('Int64')
df_books['book_ratings_count'] = pd.to_numeric(df_books['book_ratings_count'], errors='coerce').astype('Int64')

In [17]:
# Verificando os tipos das colunas do dataframe
df_books.dtypes

book_text_reviews_count      Int64
language_code               object
is_ebook                    object
book_rating                float64
book_format                 object
author_id                    Int64
num_pages                    Int64
publication_year             Int64
book_id                      int64
book_ratings_count           Int64
book_title                  object
dtype: object

#### Interações

In [18]:
df_interactions.head()

,user_id,book_id,is_read,rating,is_reviewed
0,0,948,1,5,0
1,0,947,1,5,1
2,0,946,1,5,0
3,0,945,1,5,0
4,0,944,1,5,0


In [19]:
# Verificando os tipos das colunas do dataframe
df_interactions.dtypes

user_id        int64
book_id        int64
is_read        int64
rating         int64
is_reviewed    int64
dtype: object

### Juntanto os datasets

O merge entre `df_books` e `df_genres` é no formato 'left' para garantir que todos os livros presentes em `df_books` sejam mantidos, mesmo que não haja correspondência deles em `df_genres`. Isso pode ser útil em etapas futuras, visto que mesmo sem a devida correspôndencia os dados dos livros podem conter outras informações úteis.

In [20]:
df_book_details = pd.merge(df_books, df_genres, on='book_id', how='left')

Assim como no merge realizado acima, o merge entre `df_book_details` e `df_authors` é no formato 'left' para que os dados dos livros sem autores cadastrados ainda seja mantido em etapas futuras.

In [21]:
df_authors['author_id'] = df_authors['author_id'].astype('Int64')

In [22]:
df_book_details = pd.merge(df_book_details, df_authors, on='author_id', how='left')

Por fim é realizado o merge entre `df_book_details` e `df_interactions` no formato 'inner', pois em etapas seguintes vão ser criados modelos para prever a avaliação de um livro por um determinado usuário, então é necessário que essa correnpondência esteja presente no dataframe.

In [23]:
df = pd.merge(df_book_details, df_interactions, on='book_id', how='inner')

In [24]:
df.head()

,book_text_reviews_count,language_code,is_ebook,book_rating,book_format,author_id,num_pages,publication_year,book_id,book_ratings_count,book_title,book_genre,author_rating,author_reviews_count,author_ratings_count,user_id,is_read,rating,is_reviewed
0,6,,false,3.23,Audio CD,626222,<NA>,2001,1333909,10,Good Harbor,fiction,4.08,28716.0,546796.0,38188,0,0,0
1,5,,false,3.40,,149918,<NA>,<NA>,287140,15,Runic Astrology: Starcraft and Timekeeping in ...,non-fiction,3.82,96.0,1740.0,1832,0,0,0
2,5,,false,3.40,,149918,<NA>,<NA>,287140,15,Runic Astrology: Starcraft and Timekeeping in ...,non-fiction,3.82,96.0,1740.0,2822,1,4,0
3,5,,false,3.40,,149918,<NA>,<NA>,287140,15,Runic Astrology: Starcraft and Timekeeping in ...,non-fiction,3.82,96.0,1740.0,3237,1,5,0
4,5,,false,3.40,,149918,<NA>,<NA>,287140,15,Runic Astrology: Starcraft and Timekeeping in ...,non-fiction,3.82,96.0,1740.0,3662,1,4,0


In [25]:
# Verificando o número de linhas do dataframe
df.shape[0]

4043883

In [26]:
# Verificando as colunas do dataframe
df.columns

Index(['book_text_reviews_count', 'language_code', 'is_ebook', 'book_rating',
       'book_format', 'author_id', 'num_pages', 'publication_year', 'book_id',
       'book_ratings_count', 'book_title', 'book_genre', 'author_rating',
       'author_reviews_count', 'author_ratings_count', 'user_id', 'is_read',
       'rating', 'is_reviewed'],
      dtype='object')

### Salvando o Dataframe

In [27]:
df.to_csv('data/goodreads.csv', index=False, sep=';', encoding='utf-8', header=True)